In [1]:
# necessary libraries
from datetime import date, timedelta
import gc
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

In [2]:
# load data
df_train = pd.read_csv(
    '../../../Data/favorita-grocery-sales-forecasting/train.csv', 
    usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    "../../../Data/favorita-grocery-sales-forecasting/test.csv", 
    usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

items = pd.read_csv(
    "../../../Data/favorita-grocery-sales-forecasting/items.csv",
).set_index("item_nbr")

stores = pd.read_csv(
    "../../../Data/favorita-grocery-sales-forecasting/stores.csv",
).set_index("store_nbr")

In [5]:
df_train.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2016-01-01,25,105574,2.564949,False
1,2016-01-01,25,105575,2.302585,False
2,2016-01-01,25,105857,1.386294,False
3,2016-01-01,25,108634,1.386294,False
4,2016-01-01,25,108701,1.098612,True


In [7]:
df_test.head()

id  onpromotion
store_nbr item_nbr date                              
1         96995    2017-08-16  125497040        False
          99197    2017-08-16  125497041        False
          103501   2017-08-16  125497042        False
          103520   2017-08-16  125497043        False
          103665   2017-08-16  125497044        False

In [9]:
items.head()

,family,class,perishable
item_nbr,,,
96995,GROCERY I,1093,0
99197,GROCERY I,1067,0
103501,CLEANING,3008,0
103520,GROCERY I,1028,0
103665,BREAD/BAKERY,2712,1


In [11]:
stores.head()

,city,state,type,cluster
store_nbr,,,,
1,Quito,Pichincha,D,13
2,Quito,Pichincha,D,13
3,Quito,Pichincha,D,8
4,Quito,Pichincha,D,9
5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [27]:
le = LabelEncoder()
items['family'] = le.fit_transform(items['family'].values)

stores['city'] = le.fit_transform(stores['city'].values)
stores['state'] = le.fit_transform(stores['state'].values)
stores['type'] = le.fit_transform(stores['type'].values)

In [29]:
items.head()

,family,class,perishable
item_nbr,,,
96995,12,1093,0
99197,12,1067,0
103501,7,3008,0
103520,12,1028,0
103665,5,2712,1


In [31]:
stores.head()

,city,state,type,cluster
store_nbr,,,,
1,18,12,3,13
2,18,12,3,13
3,18,12,3,8
4,18,12,3,9
5,21,14,3,4


In [56]:
df_2017 = df_train.loc[df_train.date>=pd.to_datetime('1,1,2017')]
del df_train

In [62]:
df_2017.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
35229871,2017-01-01,25,99197,0.693147,False
35229872,2017-01-01,25,103665,2.079442,False
35229873,2017-01-01,25,105574,0.693147,False
35229874,2017-01-01,25,105857,1.609438,False
35229875,2017-01-01,25,106716,1.098612,False


In [104]:
promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)

In [106]:
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)

In [116]:
del promo_2017_test, promo_2017_train

In [108]:
promo_2017_train

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False        True       False   
...                        ...         ...         ...         ...   
54        2109909        False       False       False       False   
          2110456        False       False       False       False   
          2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False        True       False       False   
...                        ...         ...         ...         ...   
54        2109909        False       False       False       False   
          2110456        False       False       False       False   
          2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-01-09  2017-01-10  ...  2017-08-06  2017-08-07  \
store_nbr item_nbr                          ...                           
1         96995          False       False  ...       False       False   
          99197          False       False  ...       False       False   
          103520         False       False  ...       False       False   
          103665         False       False  ...       False       False   
          105574         False       False  ...       False       False   
...                        ...         ...  ...         ...         ...   
54        2109909        False       False  ...       False       False   
          2110456        False       False  ...       False       False   
          2113343        False       False  ...       False       False   
          2113914        False       False  ...        True       False   
          2116416        False       False  ...       False        True   

date                2017-08-08  2017-08-09  2017-08-10  2017-08-11  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   
...                        ...         ...         ...         ...   
54        2109909        False       False       False       False   
          2110456        False       False       False       False   
          2113343        False       False       False       False   
          2113914         True        True        True       False   
          2116416        False       False       False       False   

date                2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                  
1         96995          False       False       False       False  
          99197          False       False       False       False  
          10352

In [112]:
promo_2017_test

date                2017-08-16  2017-08-17  2017-08-18  2017-08-19  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   
...                        ...         ...         ...         ...   
54        2109909        False       False       False       False   
          2110456        False       False       False       False   
          2113343        False       False       False       False   
          2113914         True        True        True        True   
          2116416        False       False       False       False   

date                2017-08-20  2017-08-21  2017-08-22  2017-08-23  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   
...                        ...         ...         ...         ...   
54        2109909        False       False       False       False   
          2110456        False       False       False       False   
          2113343        False       False       False       False   
          2113914         True        True        True        True   
          2116416        False       False       False       False   

date                2017-08-24  2017-08-25  2017-08-26  2017-08-27  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   
...                        ...         ...         ...         ...   
54        2109909        False       False       False       False   
          2110456        False       False       False        True   
          2113343        False       False       False       False   
          2113914         True        True        True        True   
          2116416        False       False       False       False   

date                2017-08-28  2017-08-29  2017-08-30  2017-08-31  
store_nbr item_nbr                                                  
1         96995          False       False       False       False  
          99197          False       False       False       False  
          103520         False       False       False       False  
          103665         False       False       False       False  
          105574         False       False       False       False  
...                        ...         ...         ...         ...  
54        2109909        False       False       False       False  
          2110456        False       False       False       False  
          2113343        False       False       False       False  
          2113914         True        True        True        True  
          2116416        False       False       False       False  

[167515 rows x 16 columns]

In [114]:
promo_2017

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False        True       False   
...                        ...         ...         ...         ...   
54        2109909        False       False       False       False   
          2110456        False       False       False       False   
          2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False        True       False       False   
...                        ...         ...         ...         ...   
54        2109909        False       False       False       False   
          2110456        False       False       False       False   
          2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-01-09  2017-01-10  ...  2017-08-22  2017-08-23  \
store_nbr item_nbr                          ...                           
1         96995          False       False  ...       False       False   
          99197          False       False  ...       False       False   
          103520         False       False  ...       False       False   
          103665         False       False  ...       False       False   
          105574         False       False  ...       False       False   
...                        ...         ...  ...         ...         ...   
54        2109909        False       False  ...       False       False   
          2110456        False       False  ...       False       False   
          2113343        False       False  ...       False       False   
          2113914        False       False  ...        True        True   
          2116416        False       False  ...       False       False   

date                2017-08-24  2017-08-25  2017-08-26  2017-08-27  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   
...                        ...         ...         ...         ...   
54        2109909        False       False       False       False   
          2110456        False       False       False        True   
          2113343        False       False       False       False   
          2113914         True        True        True        True   
          2116416        False       False       False       False   

date                2017-08-28  2017-08-29  2017-08-30  2017-08-31  
store_nbr item_nbr                                                  
1         96995          False       False       False       False  
          99197          False       False       False       False  
          10352

In [118]:
df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

In [119]:
df_2017

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995            0.0    0.000000    0.000000    0.000000   
          99197            0.0    0.000000    1.386294    0.693147   
          103520           0.0    0.693147    1.098612    0.000000   
          103665           0.0    0.000000    0.000000    1.386294   
          105574           0.0    0.000000    1.791759    2.564949   
...                        ...         ...         ...         ...   
54        2109909          0.0    0.000000    0.000000    0.000000   
          2110456          0.0    0.000000    0.000000    0.000000   
          2113343          0.0    0.000000    0.000000    0.000000   
          2113914          0.0    0.000000    0.000000    0.000000   
          2116416          0.0    0.000000    0.000000    0.000000   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.000000    0.000000   
          99197       0.693147    0.693147    1.098612    0.000000   
          103520      1.098612    1.386294    0.693147    0.000000   
          103665      1.098612    1.098612    0.693147    1.098612   
          105574      2.302585    1.945910    1.609438    1.098612   
...                        ...         ...         ...         ...   
54        2109909     0.000000    0.000000    0.000000    0.000000   
          2110456     0.000000    0.000000    0.000000    0.000000   
          2113343     0.000000    0.000000    0.000000    0.000000   
          2113914     0.000000    0.000000    0.000000    0.000000   
          2116416     0.000000    0.000000    0.000000    0.000000   

date                2017-01-09  2017-01-10  ...  2017-08-06  2017-08-07  \
store_nbr item_nbr                          ...                           
1         96995       0.000000    0.000000  ...    1.098612    1.098612   
          99197       0.000000    0.693147  ...    0.000000    1.098612   
          103520      0.693147    0.693147  ...    0.000000    0.000000   
          103665      0.000000    2.079442  ...    0.693147    1.098612   
          105574      1.386294    2.302585  ...    0.000000    1.791759   
...                        ...         ...  ...         ...         ...   
54        2109909     0.000000    0.000000  ...    0.693147    0.693147   
          2110456     0.000000    0.000000  ...    7.203406    6.481577   
          2113343     0.000000    0.000000  ...    1.098612    0.000000   
          2113914     0.000000    0.000000  ...    2.890372    0.000000   
          2116416     0.000000    0.000000  ...    0.693147    0.693147   

date                2017-08-08  2017-08-09  2017-08-10  2017-08-11  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.693147    0.000000   
          99197       0.000000    1.098612    0.000000    0.000000   
          103520      1.386294    0.000000    1.386294    0.693147   
          103665      0.000000    2.079442    2.302585    1.098612   
          105574      2.079442    1.945910    2.397895    1.791759   
...                        ...         ...         ...         ...   
54        2109909     0.000000    1.098612    0.693147    0.000000   
          2110456     6.586172    3.218876    0.000000    0.000000   
          2113343     0.000000    0.000000    0.693147    0.000000   
          2113914     2.397895    2.397895    1.609438    0.000000   
          2116416     0.000000    0.000000    0.000000    0.693147   

date                2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                  
1         96995       0.000000    0.000000    0.000000    0.000000  
          99197       0.000000    0.000000    0.000000    0.000000  
          10352

In [122]:
items = items.reindex(df_2017.index.get_level_values(1))
stores = stores.reindex(df_2017.index.get_level_values(0))

In [124]:
items

,family,class,perishable
item_nbr,,,
96995,12,1093,0
99197,12,1067,0
103520,12,1028,0
103665,5,2712,1
105574,12,1045,0
...,...,...,...
2109909,12,1074,0
2110456,3,1120,0
2113343,3,1114,0


In [126]:
stores

,city,state,type,cluster
store_nbr,,,,
1,18,12,3,13
1,18,12,3,13
1,18,12,3,13
1,18,12,3,13
1,18,12,3,13
...,...,...,...,...
54,5,10,2,3
54,5,10,2,3
54,5,10,2,3


In [146]:
df_2017_item = df_2017.groupby('item_nbr')[df_2017.columns].sum()
promo_2017_item = promo_2017.groupby('item_nbr')[promo_2017.columns].sum()